# Introdução

## Importação das bibliotecas necessarias

In [22]:
import requests
import pandas as pd
import os
import json
import sys
import matplotlib.pyplot as plt
import funcoes_comuns as fc
from importlib import reload
reload(fc)

# Dataframe base de Código do município e Ano

In [3]:
base_df = fc.get_tb_municipios()
base_df.head()

,CODIGO_MUNICIPIO,ano_nasc,CODMUNRES
0,1100015,2012,110001
1,1100015,2013,110001
2,1100015,2014,110001
3,1100015,2015,110001
4,1100015,2016,110001


# Nascidos vivos de mulheres de 2012 a 2020

In [4]:
%time
df_t_nasc_vivos = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as TOTAL_DE_NASCIDOS_VIVOS \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2020 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
...................................................(y)
Record rows length : 50268


# Quantidade de mulheres com pelo menos uma consulta de pré-natal

In [5]:
%time
df_consultas_1_pre_natal = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, \
                    count(*) as MULHERES_COM_PELO_MENOS_UMA_CONSULTA_PRENATAL \
                    FROM \"datasus-sinasc\" \
                    WHERE (ano_nasc>=2012 AND \
                    ano_nasc<=2020) AND \
                    CONSPRENAT>=1 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 0 ns
.............................................(y)
Record rows length : 44660


# Quantidade de mulheres com início do pré-natal nos três primeiros meses de gestação

In [6]:
%time
df_3mes_prenatal = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, \
                    count(*) as MULHERES_COM_INICIO_PRECOCE_DO_PRENATAL \
                    FROM \"datasus-sinasc\" \
                    WHERE (ano_nasc>=2012 AND \
                    ano_nasc<=2020) AND \
                    (MESPRENAT=1 OR MESPRENAT=2 OR MESPRENAT=3) \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs
...................................................(y)
Record rows length : 50230


# Número de mulheres com mais de sete consultas de pré-natal

In [7]:
%time
df_consultas_7_pre_natal = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, \
                    count(*) as MULHERES_COM_MAIS_DE_SETE_CONSULTAS_PRENATAL \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2020 AND \
                    CONSPRENAT>7 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 0 ns
.............................................(y)
Record rows length : 44619


# Incidência de sífilis congênita

In [9]:
dfs = pd.read_excel('Sifilis_congenita_por municipio_2012_2020.xlsx')
df_new = pd.DataFrame()
df_ano = pd.DataFrame()
df_codmunres = pd.DataFrame()

lista = []
codmunres = []
transposto = []
fc.dot_print()
for row in range(1,len(dfs.index)):
    codmunres.append(dfs.loc[row, 1])
    lista.append(dfs.loc[row, 6:14].values.tolist())
fc.dot_print()
for i in range(len(lista)):
    for j in range(len(lista[i])):
        transposto.append(lista[i][j])
fc.dot_print()
df_codmunres['CODMUNRES'] = codmunres
df_codmunres['CODMUNRES'] = df_codmunres['CODMUNRES'].astype(int)

key = ['CODMUNRES','ano_nasc']

df_ano['ano_nasc'] = [2012,2013,2014,2015,2016,2017,2018,2019,2020]
df_sifilis_congenita = pd.merge(df_codmunres.assign(key=1), df_ano.assign(key=1),on='key').drop('key', axis=1)
df_sifilis_congenita['CASOS_SC'] = transposto

...

# Finalização

In [10]:
# Conversão do CODMUNRES para inteiro, para que seja possível a junção
base_df['CODMUNRES']=base_df['CODMUNRES'].astype(int)
df_t_nasc_vivos['CODMUNRES']=df_t_nasc_vivos['CODMUNRES'].astype(int)
df_consultas_1_pre_natal['CODMUNRES']=df_consultas_1_pre_natal['CODMUNRES'].astype(int)
df_3mes_prenatal['CODMUNRES']=df_3mes_prenatal['CODMUNRES'].astype(int)
df_consultas_7_pre_natal['CODMUNRES']=df_consultas_7_pre_natal['CODMUNRES'].astype(int)
df_sifilis_congenita['CODMUNRES'] = df_sifilis_congenita['CODMUNRES'].astype(int)

# Junção em um dataframe final
df_final = pd.merge(base_df, df_t_nasc_vivos, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_consultas_1_pre_natal, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_3mes_prenatal, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_consultas_7_pre_natal, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_sifilis_congenita, how='left', on=['CODMUNRES','ano_nasc'])
# Sífilis congênita

# Preencher NaN com zeros
df_final = df_final.fillna(0)
df_final.rename(columns={'ano_nasc': 'ANO'},
               inplace=True)

del df_final['CODIGO_MUNICIPIO']

[i in range(len(df_final.columns.values.tolist()))]
aux = [i for i in range(len(df_final.columns.values.tolist()))]
aux[0] = 1
aux[1] = 0
df_final = df_final.iloc[:, aux]

In [11]:
df_final.head()

,CODMUNRES,ANO,TOTAL_DE_NASCIDOS_VIVOS,MULHERES_COM_PELO_MENOS_UMA_CONSULTA_PRENATAL,MULHERES_COM_INICIO_PRECOCE_DO_PRENATAL,MULHERES_COM_MAIS_DE_SETE_CONSULTAS_PRENATAL,CASOS_SC
0,110001,2012,386.0,381.0,277.0,156.0,0
1,110001,2013,382.0,0.0,275.0,0.0,0
2,110001,2014,341.0,338.0,245.0,146.0,0
3,110001,2015,373.0,361.0,282.0,140.0,0
4,110001,2016,326.0,321.0,252.0,147.0,0


In [12]:
df_final.columns.values.tolist()

['CODMUNRES',
 'ANO',
 'TOTAL_DE_NASCIDOS_VIVOS',
 'MULHERES_COM_PELO_MENOS_UMA_CONSULTA_PRENATAL',
 'MULHERES_COM_INICIO_PRECOCE_DO_PRENATAL',
 'MULHERES_COM_MAIS_DE_SETE_CONSULTAS_PRENATAL',
 'CASOS_SC']

In [13]:
df_final.to_csv('./indicadores_bloco3_assistencia_pre-natal_2012-2020.csv',index=False)

In [12]:
%time
df_consprenat = fc.build_df_by_sql_query("SELECT CONSULTAS, \
                    count(*) as TOTAL_CONSULTAS_2013 \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc=2013 \
                    GROUP BY CONSULTAS")

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 8.82 µs
.(y)
Record rows length : 5


In [13]:
df_consprenat.head(30)

,CONSULTAS,TOTAL_CONSULTAS_2013
0,1,77906
1,2,212293
2,3,769261
3,4,1812681
4,9,31886


In [7]:
%time
sql = {
          "token": {
            "token": fc.token
          },
          "sql": {
            "sql": {"query":'describe "datasus-sinasc"'}
          }
        }
resp = requests.post(os.path.join(fc.url_base,'sql_query'), json=sql)
fields = [x[0] for x in resp.json()['rows']]
print(fields)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 8.58 µs
['APGAR1', 'APGAR5', 'CODANOMAL', 'CODBAINASC', 'CODBAIRES', 'CODCART', 'CODESTAB', 'CODINST', 'CODMUNCART', 'CODMUNNASC', 'CODMUNNATU', 'CODMUNRES', 'CODOCUPMAE', 'CODPAISRES', 'CODUFNATU', 'CONSPRENAT', 'CONSULTAS', 'CONTADOR', 'DIFDATA', 'DTCADASTRO', 'DTDECLARAC', 'DTNASC', 'DTNASCMAE', 'DTRECEBIM', 'DTRECORIG', 'DTRECORIGA', 'DTREGCART', 'DTULTMENST', 'ESCMAE', 'ESCMAE2010', 'ESCMAEAGR1', 'ESTCIVMAE', 'GESTACAO', 'GRAVIDEZ', 'HORANASC', 'IDADEMAE', 'IDADEPAI', 'IDANOMAL', 'KOTELCHUCK', 'LOCNASC', 'MESPRENAT', 'NATURALMAE', 'NOVO', 'NUMERODN', 'NUMERODV', 'NUMEROLOTE', 'NUMREGCART', 'ORIGEM', 'PARIDADE', 'PARTO', 'PESO', 'PREFIXODN', 'QTDFILMORT', 'QTDFILVIVO', 'QTDGESTANT', 'QTDPARTCES', 'QTDPARTNOR', 'RACACOR', 'RACACORMAE', 'RACACORN', 'RACACOR_RN', 'SEMAGESTAC', 'SERIESCMAE', 'SEXO', 'STCESPARTO', 'STDNEPIDEM', 'STDNNOVA', 'STTRABPART', 'TPAPRESENT', 'TPDOCRESP', 'TPFUNCRESP', 'TPMETESTIM', 'TPNASCASSI', 'TPROB